In [85]:
# import libraries
import os
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from pathlib import Path
from directories import *
import numpy as np
from parameters_config import Config
import matplotlib.pyplot as plt
import seaborn as sns

print(BASE_DIRECTORY.absolute())

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from category_encoders import OneHotEncoder, TargetEncoder
from sklearn.impute import SimpleImputer,KNNImputer

/mnt/data/boses08/hype-prediction-longitudinal/ml/v2/src/Suparno Experiment


In [3]:
BASE_PATH='/home/boses08/hype-prediction-longitudinal/ml'

# DATA_FILE_PATH="../../data/raw/"
DATA_FILE_PATH = "/home/boses08/hype-prediction-longitudinal/ml/data/raw_with_mrn/"
DATA_UNSUPERVISED_PATH = "/home/boses08/hype-prediction-longitudinal/ml/v2/prepare_multi_modal_data/data/diag_drug_proc_365.parquet"
MRN_PATH = "/home/boses08/hype-prediction-longitudinal/ml/v2/src/Suparno Experiment/data/selected_MRNs/ML_dataset_365.pkl"

DATA_FILE_PATH_PROCESS="/home/boses08/hype-prediction-longitudinal/ml/v2/src/Suparno Experiment/data/processed/"


In [4]:
NO_OF_SUBSETS='all'
version='v2'
DATA_FILE_PATH_PROCESS=BASE_PATH+"/data/"+version+"/processed/"
# DATA_FILE_PATH_PROCESS = '/mnt/hype_long/data'
print(DATA_FILE_PATH_PROCESS)
WEIGHTS_DIR=BASE_PATH+'/'+version+'/model_parameters/initial_weights_'+str(NO_OF_SUBSETS)
print(WEIGHTS_DIR)
initial_weights=WEIGHTS_DIR +'/initial_weights'
initial_weights

/home/boses08/hype-prediction-longitudinal/ml/data/v2/processed/
/home/boses08/hype-prediction-longitudinal/ml/v2/model_parameters/initial_weights_all


'/home/boses08/hype-prediction-longitudinal/ml/v2/model_parameters/initial_weights_all/initial_weights'

In [5]:
def train_test_split(df, test_df_control):
    train_df = df[df.train_test == 'train']
    test_df = df[df.train_test == 'test']

    if test_df_control_ratio is not None:
        test_df_ht = test_df[test_df.HT == '1']
        test_df_not_ht = test_df[test_df.HT == '0']
        test_df_not_ht = test_df_not_ht.sample(test_df_control_ratio * test_df_ht.shape[0], random_state=42)
        test_df = pd.concat([test_df_ht,test_df_not_ht])
    
    mrn_train = set(train_df['medical_record_number'])
    mrn_test = set(test_df['medical_record_number'])
    train_test_common = mrn_train.intersection(mrn_test)
    train_df = train_df.drop(columns=['train_test', 'address_zip'])
    test_df = test_df.drop(columns=['train_test', 'address_zip'])
    return train_df, test_df


In [6]:
NA_removal_threshold = 80  ##atleast this many columns should have a non null value
test_df_control_ratio = 2
df =  pd.read_pickle(MRN_PATH)
#Dataset_C['MRN'] = Dataset_C.index
##Dataset_C.reset_index(level=0, inplace=True)
df = df.dropna(axis=0, thresh = NA_removal_threshold)
print("final dataframe shape after dropping NAs" + str(df.shape))
print(pd.crosstab(df.train_test, df.HT))
MRN_train_df, MRN_test_df = train_test_split(df, test_df_control_ratio)

final dataframe shape after dropping NAs(266670, 316)
HT               0      1
train_test               
test        153245   3269
train        76528  33628


In [7]:
print(MRN_test_df.HT.value_counts())

0    6538
1    3269
Name: HT, dtype: int64


In [8]:
train_mrn = list(MRN_train_df.medical_record_number)
test_mrn = list(MRN_test_df.medical_record_number)

In [9]:
def load_sequence_data():
    with open(DATA_FILE_PATH+"data.txt", "rb") as fp:
        X = pickle.load(fp)

    with open(DATA_FILE_PATH+ "label.txt", "rb") as fp:
        y = pickle.load(fp)

    df = pd.concat([pd.DataFrame(X, columns=['medical_record_number','sequence']), pd.DataFrame(y, columns=['target'])], axis=1)
    return df

def load_unsupervised_data(path):
    df = pd.read_parquet(path)
#     df = pd.read_pickle(path)
    return df

def filter_by_mrn(df,mrn_list):
    return df[df.medical_record_number.isin(mrn_list)]


In [10]:
sequence_data = load_sequence_data()

In [11]:
train_sequence_data = filter_by_mrn(sequence_data, train_mrn)
test_sequence_data = filter_by_mrn(sequence_data, test_mrn)

In [12]:
print(train_sequence_data.shape)
print(test_sequence_data.shape)

(110156, 3)
(9807, 3)


In [13]:
train_sequence_data = train_sequence_data.drop(['medical_record_number','target'], axis = 1)
test_sequence_data = test_sequence_data.drop(['medical_record_number','target'], axis = 1)

train_sequence_data = np.asarray(train_sequence_data['sequence'])
test_sequence_data = np.asarray(test_sequence_data['sequence'])

In [14]:
train_sequence_data = tf.keras.preprocessing.sequence.pad_sequences(train_sequence_data, maxlen=Config.MAX_REVIEW_LENGTH, padding='pre', truncating='post')
test_sequence_data = tf.keras.preprocessing.sequence.pad_sequences(test_sequence_data, maxlen=Config.MAX_REVIEW_LENGTH, padding='pre', truncating='post')

print('After padding the sequence with the longest length the shape is:',train_sequence_data.shape)

print(train_sequence_data.max())
print(test_sequence_data.max())

After padding the sequence with the longest length the shape is: (110156, 150)
340
340


In [15]:
unsupervised_data = load_unsupervised_data(DATA_UNSUPERVISED_PATH)

In [16]:
train_unsupervised_data = filter_by_mrn(unsupervised_data, train_mrn)
test_unsupervised_data = filter_by_mrn(unsupervised_data, test_mrn)

In [17]:
len(train_unsupervised_data.columns)

128

In [18]:
numerical_cols = [c for c in train_unsupervised_data.columns if train_unsupervised_data[c].dtype in [np.float, np.int] and c not in ['HT']]
categorical_cols = [c for c in train_unsupervised_data.columns if train_unsupervised_data[c].dtype in [np.object] and c not in ['HT']]

print(len(numerical_cols)+len(categorical_cols))


127


In [19]:
train_labels = train_unsupervised_data.pop('HT').astype('int')
test_labels = test_unsupervised_data.pop('HT').astype('int')

train_unsupervised_data.drop(['medical_record_number'],inplace=True, axis = 1)
test_unsupervised_data.drop(['medical_record_number'],inplace=True, axis = 1)

/home/boses08/.local/lib/python3.8/site-packages/pandas/core/frame.py:4160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [28]:
from sklearn.pipeline import Pipeline
def impute_unsupervised_data(train_df, test_df):
    categorical_cols = [c for c in train_df.columns if train_df[c].dtype in [np.object] and c not in ['HT']]
    numerical_cols = [c for c in train_df.columns if train_df[c].dtype in [np.float, np.int] and c not in ['HT']]
    print("Number of categorical features " + str(len(categorical_cols)) + " and number of numerical features "+ str(len(numerical_cols)))
    
    ct = Pipeline([('ct',
        ColumnTransformer([
        ('num', StandardScaler(), numerical_cols),
        ('cat', TargetEncoder(drop_invariant = True, handle_missing = 'return_nan', min_samples_leaf = 10), categorical_cols)])),
        ('imputer' , SimpleImputer(missing_values=np.nan, strategy='median'))])

    fitted_train= ct.fit(train_df,pd.to_numeric(train_labels))
    
    train_df = fitted_train.transform(train_df)
    test_df = fitted_train.transform(test_df)
    
    return train_df,test_df

In [29]:
train_unsupervised_data, test_unsupervised_data = impute_unsupervised_data(train_unsupervised_data,
                                                                           test_unsupervised_data)

Number of categorical features 26 and number of numerical features 100


/home/boses08/.local/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [30]:
print(np.isnan(train_unsupervised_data).any())
print(np.isnan(test_unsupervised_data).any())

False
False


## Training and evaluating the LSTM + AUX multi input model

In [31]:
np.random.seed(7)
tf.random.set_seed(seed=7)

# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [32]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

In [33]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import concatenate,Dense
from keras.models import Model,load_model

main_input = keras.Input(shape=(train_sequence_data.shape[1],), name='main_input') # dtype='int32'

# This embedding layer will encode the input sequence
# into a sequence of dense 512-dimensional vectors.
x = layers.Embedding(Config.VOCAB_SIZE, Config.EMBEDDING_DIM, input_length=Config.MAX_REVIEW_LENGTH, name='Embedding_1')(main_input)

# A LSTM will transform the vector sequence into a single vector,
# containing information about the entire sequence
lstm_out = layers.LSTM(100, name='lstm_1', dropout=0.5)(x)

aux_input=keras.Input(shape=(train_unsupervised_data.shape[1],),name='aux_input')

# We concatenate the lstm output to auxillary input
x = concatenate([lstm_out, aux_input])

# And finally we add the main logistic regression layer
main_output = Dense(1, activation='sigmoid', name='main_output')(x)

model = Model(inputs=[main_input, aux_input], outputs=[main_output])

print(model.summary())

# compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=Config.METRICS)


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         [(None, 150)]        0                                            
__________________________________________________________________________________________________
Embedding_1 (Embedding)         (None, 150, 100)     34100       main_input[0][0]                 
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 100)          80400       Embedding_1[0][0]                
__________________________________________________________________________________________________
aux_input (InputLayer)          [(None, 121)]        0                                            
_______________________________________________________________________________________

In [47]:
# # Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_average_precision', 
    verbose=1,
    patience=20,
    mode='max',
    restore_best_weights=True)

history = model.fit({'main_input': train_sequence_data, 'aux_input': train_unsupervised_data},
          {'main_output': train_labels},
          epochs=200,validation_split=0.3,
#           validation_data=([val_features_sequence,val_features_unsupervised],val_labels),
          batch_size=2048, verbose=1, callbacks=[early_stopping, tensorboard_callback])

Epoch 1/200
 1/38 [..............................] - ETA: 0s - loss: 0.7104 - tp: 550.0000 - fp: 834.0000 - tn: 583.0000 - fn: 81.0000 - accuracy: 0.5532 - precision: 0.3974 - recall: 0.8716 - auc: 0.6799 - f1_score: 0.5459 - average_precision: 0.3974WARNING:tensorflow:From /home/boses08/.local/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
38/38 [==============================] - 13s 353ms/step - loss: 0.5394 - tp: 9415.0000 - fp: 7846.0000 - tn: 45677.0000 - fn: 14171.0000 - accuracy: 0.7145 - precision: 0.5454 - recall: 0.3992 - auc: 0.7430 - f1_score: 0.4164 - average_precision: 0.5285 - val_loss: 0.4563 - val_tp: 5682.0000 - val_fp: 3015.0000 - val_tn: 19990.0000 - val_fn: 4360.0000 - val_accuracy: 0.7768 - val_precision: 0.6533 - val_recall: 0.5658 - val_auc: 0.8367 - val_f1_score: 0.609

38/38 [==============================] - 12s 322ms/step - loss: 0.2439 - tp: 19574.0000 - fp: 3011.0000 - tn: 50512.0000 - fn: 4012.0000 - accuracy: 0.9089 - precision: 0.8667 - recall: 0.8299 - auc: 0.9652 - f1_score: 0.8477 - average_precision: 0.8674 - val_loss: 0.2442 - val_tp: 8242.0000 - val_fp: 1147.0000 - val_tn: 21858.0000 - val_fn: 1800.0000 - val_accuracy: 0.9108 - val_precision: 0.8778 - val_recall: 0.8208 - val_auc: 0.9652 - val_f1_score: 0.8504 - val_average_precision: 0.8789
Epoch 33/200
38/38 [==============================] - 12s 322ms/step - loss: 0.2425 - tp: 19578.0000 - fp: 2936.0000 - tn: 50587.0000 - fn: 4008.0000 - accuracy: 0.9099 - precision: 0.8696 - recall: 0.8301 - auc: 0.9658 - f1_score: 0.8492 - average_precision: 0.8708 - val_loss: 0.2428 - val_tp: 8465.0000 - val_fp: 1282.0000 - val_tn: 21723.0000 - val_fn: 1577.0000 - val_accuracy: 0.9135 - val_precision: 0.8685 - val_recall: 0.8430 - val_auc: 0.9664 - val_f1_score: 0.8570 - val_average_precision: 0.86

In [83]:
model.metrics_names

['loss',
 'tp',
 'fp',
 'tn',
 'fn',
 'accuracy',
 'precision',
 'recall',
 'auc',
 'f1_score',
 'average_precision']

In [82]:
print(history.history['val_loss'][55])
print(history.history['val_tp'][55])
print(history.history['val_fp'][55])
print(history.history['val_tn'][55])
print(history.history['val_fn'][55])
print(history.history['val_accuracy'][55])
print(history.history['val_precision'][55])
print(history.history['val_recall'][55])
print(history.history['val_auc'][55])
print(history.history['val_f1_score'][55])
print(history.history['val_average_precision'][55])

0.20011594891548157
8815.0
997.0
22008.0
1227.0
0.9327018857002258
0.8983896970748901
0.8778131604194641
0.9793612360954285
0.8890920877456665
0.900587260723114


In [54]:
# Testing the model on test data
results = model.evaluate([test_sequence_data,test_unsupervised_data],
                         test_labels, batch_size=2048, verbose=1)

5/5 [==============================] - 0s 73ms/step - loss: 0.2945 - tp: 2641.0000 - fp: 598.0000 - tn: 5940.0000 - fn: 628.0000 - accuracy: 0.8750 - precision: 0.8154 - recall: 0.8079 - auc: 0.9448 - f1_score: 0.8117 - average_precision: 0.8155


In [52]:
model.metrics_names

['loss',
 'tp',
 'fp',
 'tn',
 'fn',
 'accuracy',
 'precision',
 'recall',
 'auc',
 'f1_score',
 'average_precision']

In [55]:
print(results)

[0.2944844961166382, 2641.0, 598.0, 5940.0, 628.0, 0.8749872446060181, 0.8153750896453857, 0.8078923225402832, 0.9447919726371765, 0.8117319345474243, 0.8154597282409668]


In [58]:
# Predicting probability of different output classes

predicted_proba = model.predict({'main_input': test_sequence_data, 'aux_input': test_unsupervised_data})

[[0.09315826]
 [0.01751389]
 [0.7259559 ]
 ...
 [0.4219809 ]
 [0.7186493 ]
 [0.06293106]]


In [64]:
print(np.array(test_labels).reshape(9807,1))

[[0]
 [0]
 [1]
 ...
 [0]
 [0]
 [0]]


In [65]:
y_pred_proba = np.append(predicted_proba, np.array(test_labels).reshape(9807,1), axis=1)

In [68]:
prob_df = pd.DataFrame(columns = ['probability','classes'],data = y_pred_proba)

In [71]:
prob_df.to_csv('/home/boses08/hype-prediction-longitudinal/ml/v2/src/Suparno Experiment/Class_prediction_probability_test.csv')

In [ ]:
## TODO

result of model.predict_proba --> test_class

only main input with LSTM

only aux data with LSTM



## Training and evaluating the LSTM model with only sequence data

In [129]:
from model_template import make_model

In [139]:
# model = make_model(Config.VOCAB_SIZE,
#                    Config.EMBEDDING_DIM,
#                    Config.MAX_REVIEW_LENGTH,
#                    Config.METRICS)

model = tf.keras.Sequential()
model.add(
    tf.keras.layers.Embedding(
        Config.VOCAB_SIZE, Config.EMBEDDING_DIM, input_length=Config.MAX_REVIEW_LENGTH, name="em_1"
    )
)
model.add(tf.keras.layers.LSTM(100, name="lstm_1", dropout=0.5, return_sequences=True))
model.add(tf.keras.layers.LSTM(100, name="lstm_2"))
model.add(
    tf.keras.layers.Dense(
        1, activation="sigmoid", name="output"
    )
)

print(model.summary())

# compile the model
model.compile(
        optimizer=tf.keras.optimizers.Adam(lr=1e-3),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=Config.METRICS,
)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
em_1 (Embedding)             (None, 150, 100)          34100     
_________________________________________________________________
lstm_1 (LSTM)                (None, 150, 100)          80400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
output (Dense)               (None, 1)                 101       
Total params: 195,001
Trainable params: 195,001
Non-trainable params: 0
_________________________________________________________________
None


In [140]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs/only_LSTM")

In [141]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_average_precision', 
    verbose=1,
    patience=20,
    mode='max',
    restore_best_weights=True)

history1 = model.fit(train_sequence_data,train_labels,
          epochs=200,validation_split=0.3,
          batch_size=2048, verbose=1, callbacks=[early_stopping, tensorboard_callback])

Epoch 1/200
38/38 [==============================] - 23s 595ms/step - loss: 0.5365 - tp: 9025.0000 - fp: 5847.0000 - tn: 54214.0000 - fn: 17830.0000 - accuracy: 0.7276 - precision: 0.6068 - recall: 0.3361 - auc: 0.7449 - f1_score: 0.3494 - average_precision: 0.4560 - val_loss: 0.4720 - val_tp: 5700.0000 - val_fp: 3408.0000 - val_tn: 19597.0000 - val_fn: 4342.0000 - val_accuracy: 0.7655 - val_precision: 0.6258 - val_recall: 0.5676 - val_auc: 0.8169 - val_f1_score: 0.5990 - val_average_precision: 0.6303
Epoch 2/200
38/38 [==============================] - 21s 558ms/step - loss: 0.4665 - tp: 12870.0000 - fp: 6911.0000 - tn: 46612.0000 - fn: 10716.0000 - accuracy: 0.7714 - precision: 0.6506 - recall: 0.5457 - auc: 0.8223 - f1_score: 0.5925 - average_precision: 0.6514 - val_loss: 0.4531 - val_tp: 5872.0000 - val_fp: 3016.0000 - val_tn: 19989.0000 - val_fn: 4170.0000 - val_accuracy: 0.7826 - val_precision: 0.6607 - val_recall: 0.5847 - val_auc: 0.8344 - val_f1_score: 0.6254 - val_average_pre

38/38 [==============================] - 21s 559ms/step - loss: 0.4325 - tp: 14196.0000 - fp: 6682.0000 - tn: 46841.0000 - fn: 9390.0000 - accuracy: 0.7916 - precision: 0.6800 - recall: 0.6019 - auc: 0.8511 - f1_score: 0.6382 - average_precision: 0.6811 - val_loss: 0.4316 - val_tp: 5977.0000 - val_fp: 2782.0000 - val_tn: 20223.0000 - val_fn: 4065.0000 - val_accuracy: 0.7928 - val_precision: 0.6824 - val_recall: 0.5952 - val_auc: 0.8521 - val_f1_score: 0.6391 - val_average_precision: 0.6893
Epoch 18/200
38/38 [==============================] - 21s 561ms/step - loss: 0.4314 - tp: 14153.0000 - fp: 6487.0000 - tn: 47036.0000 - fn: 9433.0000 - accuracy: 0.7935 - precision: 0.6857 - recall: 0.6001 - auc: 0.8521 - f1_score: 0.6395 - average_precision: 0.6864 - val_loss: 0.4292 - val_tp: 6213.0000 - val_fp: 3011.0000 - val_tn: 19994.0000 - val_fn: 3829.0000 - val_accuracy: 0.7930 - val_precision: 0.6736 - val_recall: 0.6187 - val_auc: 0.8536 - val_f1_score: 0.6487 - val_average_precision: 0.68

38/38 [==============================] - 21s 557ms/step - loss: 0.4185 - tp: 14405.0000 - fp: 6384.0000 - tn: 47139.0000 - fn: 9181.0000 - accuracy: 0.7981 - precision: 0.6929 - recall: 0.6107 - auc: 0.8616 - f1_score: 0.6492 - average_precision: 0.6938 - val_loss: 0.4210 - val_tp: 5840.0000 - val_fp: 2582.0000 - val_tn: 20423.0000 - val_fn: 4202.0000 - val_accuracy: 0.7947 - val_precision: 0.6934 - val_recall: 0.5816 - val_auc: 0.8590 - val_f1_score: 0.6359 - val_average_precision: 0.6996
Epoch 34/200
38/38 [==============================] - 21s 561ms/step - loss: 0.4167 - tp: 14435.0000 - fp: 6194.0000 - tn: 47329.0000 - fn: 9151.0000 - accuracy: 0.8010 - precision: 0.6997 - recall: 0.6120 - auc: 0.8632 - f1_score: 0.6525 - average_precision: 0.7009 - val_loss: 0.4225 - val_tp: 6099.0000 - val_fp: 2749.0000 - val_tn: 20256.0000 - val_fn: 3943.0000 - val_accuracy: 0.7975 - val_precision: 0.6893 - val_recall: 0.6073 - val_auc: 0.8586 - val_f1_score: 0.6508 - val_average_precision: 0.69

38/38 [==============================] - 21s 561ms/step - loss: 0.4043 - tp: 14852.0000 - fp: 6116.0000 - tn: 47407.0000 - fn: 8734.0000 - accuracy: 0.8074 - precision: 0.7083 - recall: 0.6297 - auc: 0.8720 - f1_score: 0.6662 - average_precision: 0.7096 - val_loss: 0.4159 - val_tp: 6300.0000 - val_fp: 2861.0000 - val_tn: 20144.0000 - val_fn: 3742.0000 - val_accuracy: 0.8002 - val_precision: 0.6877 - val_recall: 0.6274 - val_auc: 0.8639 - val_f1_score: 0.6604 - val_average_precision: 0.6943
Epoch 50/200
38/38 [==============================] - 21s 560ms/step - loss: 0.4032 - tp: 14956.0000 - fp: 6293.0000 - tn: 47230.0000 - fn: 8630.0000 - accuracy: 0.8065 - precision: 0.7038 - recall: 0.6341 - auc: 0.8726 - f1_score: 0.6668 - average_precision: 0.7047 - val_loss: 0.4175 - val_tp: 6037.0000 - val_fp: 2596.0000 - val_tn: 20409.0000 - val_fn: 4005.0000 - val_accuracy: 0.8003 - val_precision: 0.6993 - val_recall: 0.6012 - val_auc: 0.8627 - val_f1_score: 0.6511 - val_average_precision: 0.70

38/38 [==============================] - 21s 561ms/step - loss: 0.3912 - tp: 15137.0000 - fp: 5973.0000 - tn: 47550.0000 - fn: 8449.0000 - accuracy: 0.8130 - precision: 0.7171 - recall: 0.6418 - auc: 0.8808 - f1_score: 0.6763 - average_precision: 0.7182 - val_loss: 0.4165 - val_tp: 5995.0000 - val_fp: 2594.0000 - val_tn: 20411.0000 - val_fn: 4047.0000 - val_accuracy: 0.7990 - val_precision: 0.6980 - val_recall: 0.5970 - val_auc: 0.8637 - val_f1_score: 0.6489 - val_average_precision: 0.7063
Epoch 66/200
38/38 [==============================] - 21s 561ms/step - loss: 0.3899 - tp: 15266.0000 - fp: 5932.0000 - tn: 47591.0000 - fn: 8320.0000 - accuracy: 0.8152 - precision: 0.7202 - recall: 0.6472 - auc: 0.8817 - f1_score: 0.6817 - average_precision: 0.7212 - val_loss: 0.4219 - val_tp: 6553.0000 - val_fp: 3318.0000 - val_tn: 19687.0000 - val_fn: 3489.0000 - val_accuracy: 0.7940 - val_precision: 0.6639 - val_recall: 0.6526 - val_auc: 0.8599 - val_f1_score: 0.6625 - val_average_precision: 0.67

In [142]:
print('validation score:')
val_score = {
    'val_loss':history1.history['val_loss'][78],'val_tp':history1.history['val_tp'][78],
    'val_fp':history1.history['val_fp'][78],'val_tn':history1.history['val_tn'][78],
    'val_fn':history1.history['val_fn'][78],'val_accuracy':history1.history['val_accuracy'][78],
    'val_precision':history1.history['val_precision'][78],'val_recall':history1.history['val_recall'][78],
    'val_auc':history1.history['val_auc'][78],'val_f1_score':history1.history['val_f1_score'][78],
    'val_average_precision':history1.history['val_average_precision'][78]
}
pd.DataFrame.from_dict(val_score, orient='index')

validation score:


,0
val_loss,0.423817
val_tp,6247.000000
val_fp,2913.000000
val_tn,20092.000000
val_fn,3795.000000
val_accuracy,0.797016
val_precision,0.681987
val_recall,0.622087
val_auc,0.861331
val_f1_score,0.655933


In [143]:
# Testing the model on test data
results = model.evaluate(test_sequence_data,test_labels, batch_size=2048, verbose=1)


5/5 [==============================] - 1s 117ms/step - loss: 0.5254 - tp: 1376.0000 - fp: 587.0000 - tn: 5951.0000 - fn: 1893.0000 - accuracy: 0.7471 - precision: 0.7010 - recall: 0.4209 - auc: 0.8096 - f1_score: 0.5261 - average_precision: 0.7012


In [144]:
print('Result on test data:')
result = {'metrics':model.metrics_names, 'scores': results}
pd.DataFrame(result, columns=['metrics','scores'])

Result on test data:


,metrics,scores
0,loss,0.525374
1,tp,1376.000000
2,fp,587.000000
3,tn,5951.000000
4,fn,1893.000000
5,accuracy,0.747119
6,precision,0.700968
7,recall,0.420924
8,auc,0.809648
9,f1_score,0.526105


## Training and evaluating the Fully connected NN model with only aux data

In [168]:
from keras.layers import Dropout
from keras import regularizers

# define the keras model
model = tf.keras.Sequential()
model.add(Dense(5, input_dim=train_unsupervised_data.shape[1], activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))



print(model.summary())

    # compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=Config.METRICS,
)

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 5)                 610       
_________________________________________________________________
dropout_4 (Dropout)          (None, 5)                 0         
_________________________________________________________________
dense_31 (Dense)             (None, 10)                60        
_________________________________________________________________
dropout_5 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 11        
Total params: 681
Trainable params: 681
Non-trainable params: 0
_________________________________________________________________
None


In [169]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs/only_AUX")
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_average_precision', 
    verbose=1,
    patience=5,
    mode='max',
    restore_best_weights=True)

history1 = model.fit(train_unsupervised_data,train_labels,
          epochs=30,validation_split=0.2,
          batch_size=128, verbose=1, callbacks=[early_stopping, tensorboard_callback])

Epoch 1/30
689/689 [==============================] - 5s 8ms/step - loss: 0.5865 - tp: 7626.0000 - fp: 6398.0000 - tn: 61263.0000 - fn: 22644.0000 - accuracy: 0.7034 - precision: 0.5438 - recall: 0.2519 - auc: 0.7032 - f1_score: 0.2207 - average_precision: 0.4301 - val_loss: 0.3304 - val_tp: 4605.0000 - val_fp: 55.0000 - val_tn: 15350.0000 - val_fn: 2022.0000 - val_accuracy: 0.9057 - val_precision: 0.9882 - val_recall: 0.6949 - val_auc: 0.9611 - val_f1_score: 0.8156 - val_average_precision: 0.9884
Epoch 2/30
689/689 [==============================] - 4s 6ms/step - loss: 0.3991 - tp: 13982.0000 - fp: 1154.0000 - tn: 59969.0000 - fn: 13019.0000 - accuracy: 0.8392 - precision: 0.9238 - recall: 0.5178 - auc: 0.8733 - f1_score: 0.6607 - average_precision: 0.9274 - val_loss: 0.2409 - val_tp: 4896.0000 - val_fp: 63.0000 - val_tn: 15342.0000 - val_fn: 1731.0000 - val_accuracy: 0.9186 - val_precision: 0.9873 - val_recall: 0.7388 - val_auc: 0.9673 - val_f1_score: 0.8447 - val_average_precision: 

In [164]:
print('validation score:')
val_score = {
    'val_loss':history1.history['val_loss'][7],'val_tp':history1.history['val_tp'][7],
    'val_fp':history1.history['val_fp'][7],'val_tn':history1.history['val_tn'][7],
    'val_fn':history1.history['val_fn'][7],'val_accuracy':history1.history['val_accuracy'][7],
    'val_precision':history1.history['val_precision'][7],'val_recall':history1.history['val_recall'][7],
    'val_auc':history1.history['val_auc'][7],'val_f1_score':history1.history['val_f1_score'][7],
    'val_average_precision':history1.history['val_average_precision'][7]
}
pd.DataFrame.from_dict(val_score, orient='index')

validation score:


,0
val_loss,0.147426
val_tp,6104.000000
val_fp,585.000000
val_tn,14820.000000
val_fn,523.000000
val_accuracy,0.949710
val_precision,0.912543
val_recall,0.921080
val_auc,0.989330
val_f1_score,0.916346


In [165]:
# Testing the model on test data
results = model.evaluate(test_unsupervised_data,test_labels, batch_size=256, verbose=1)

39/39 [==============================] - 0s 4ms/step - loss: 0.6415 - tp: 3221.0000 - fp: 3915.0000 - tn: 2623.0000 - fn: 48.0000 - accuracy: 0.5959 - precision: 0.4514 - recall: 0.9853 - auc: 0.9017 - f1_score: 0.6184 - average_precision: 0.4517


In [166]:
print('Result on test data:')
result = {'metrics':model.metrics_names, 'scores': results}
pd.DataFrame(result, columns=['metrics','scores'])

Result on test data:


,metrics,scores
0,loss,0.641508
1,tp,3221.000000
2,fp,3915.000000
3,tn,2623.000000
4,fn,48.000000
5,accuracy,0.595901
6,precision,0.451373
7,recall,0.985317
8,auc,0.901682
9,f1_score,0.618395
